In [1]:
!pip3 install psycopg2

Defaulting to user installation because normal site-packages is not writeable
  Using cached psycopg2-2.9.9.tar.gz (384 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running egg_info
      creating /private/var/folders/cr/p0cmx4x1377334drprykfmrw0000gn/T/pip-pip-egg-info-8xd6dp_3/psycopg2.egg-info
      writing /private/var/folders/cr/p0cmx4x1377334drprykfmrw0000gn/T/pip-pip-egg-info-8xd6dp_3/psycopg2.egg-info/PKG-INFO
      writing dependency_links to /private/var/folders/cr/p0cmx4x1377334drprykfmrw0000gn/T/pip-pip-egg-info-8xd6dp_3/psycopg2.egg-info/dependency_links.txt
      writing top-level names to /private/var/folders/cr/p0cmx4x1377334drprykfmrw0000gn/T/pip-pip-egg-info-8xd6dp_3/psycopg2.egg-info/top_level.txt
      writing manifest file '/private/var/folders/cr/p0cmx4x1377334drprykfmrw0000gn/T/pip-pip-egg-info-8xd6dp_3/psycopg2.egg-in

In [9]:
import requests
import base64

def get_access_token_from_code(auth_code):
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    response = requests.post("https://accounts.spotify.com/api/token", data={
        'grant_type': 'authorization_code',
        'code': auth_code,
        'redirect_uri': REDIRECT_URI
    }, headers={
        "Authorization": f"Basic {auth_header}"
    })
    return response.json().get("access_token"), response.json().get("refresh_token")

def refresh_access_token(refresh_token):
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    response = requests.post("https://accounts.spotify.com/api/token", data={
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }, headers={
        "Authorization": f"Basic {auth_header}"
    })
    return response.json().get("access_token")
CLIENT_ID = '43c8714e0d8e4d56a49e5eaf3467802c'
CLIENT_SECRET = 'cc7e7aec4b2948ae96e891b6554d3dd0'
REDIRECT_URI = 'http://localhost:8880/callback'
AUTHORIZATION_CODE = 'AQBKugh-AXwpETZ2PHqasSKQyjfAIrPIhsMSiITyZVFDRPJae7QsjzpUTxnH_ygiKAE1WkTIUfhPOiHGLva41uZhaqQEGmMOEIf3P3fr0xF7rdb8E1rESu2AaNZoZjNSs2vf5_-Nc1Z7RrGJ2qagc6Gxc0MVc8qMVgOy-gIGdrThuZsYiEic2n4QVHAgqud4ZfIur6rqkCSY7lGj2RammLulaFdwPZGHvSCdrdylD2Fn9eK7wMAMGwDZ9aqJ2r3dUWLYudz6PVHHvXk7'
access_token, refresh_token = get_access_token_from_code(AUTHORIZATION_CODE)

headers = {
    "Authorization": f"Bearer {access_token}"
}

#https://accounts.spotify.com/en/authorize?client_id=43c8714e0d8e4d56a49e5eaf3467802c&response_type=code&redirect_uri=http://localhost:8880/callback&scope=user-top-read%20user-read-private%20playlist-read-private%20user-read-email

In [10]:
headers["Authorization"] = f"Bearer {access_token}"
response = requests.get("https://api.spotify.com/v1/me/top/tracks?time_range=medium_term&limit=30", headers=headers)

In [8]:
from flask import Flask, render_template, redirect, session, make_response, request, url_for
from flask_sqlalchemy import SQLAlchemy
from urllib.parse import urlencode
import os
import requests
import logging
from pathlib import Path

app = Flask(__name__)
app.secret_key = os.urandom(24)
app.config.from_pyfile('config.py')
BASE_DIR = Path(os.getcwd())
DB_PATH = BASE_DIR / 'db' / 'songs.db'
app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{DB_PATH}'


db = SQLAlchemy(app)
class Song(db.Model):
    uid = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(255), nullable=False)
    artist = db.Column(db.String(255), nullable=False)
with app.app_context():
    db.create_all()

def getToken(code):
    token_url = 'https://accounts.spotify.com/api/token'
    authorization = app.config['AUTHORIZATION']
    redirect_uri = app.config['REDIRECT_URI']
    headers = {'Authorization': f"Basic {authorization}",  # Updated the Authorization header
               'Accept': 'application/json',
               'Content-Type': 'application/x-www-form-urlencoded'}
    body = {'code': code, 'redirect_uri': redirect_uri,
            'grant_type': 'authorization_code'}
    post_response = requests.post(token_url, headers=headers, data=body)
    if post_response.status_code == 200:
        pr = post_response.json()
        return pr['access_token'], pr['refresh_token'], pr['expires_in']
    else:
        logging.error('getToken:' + str(post_response.status_code))
        return None
def song_id(song_title, artist_name):
    headers = {'Authorization': f"Bearer {access_token}"}
    response = requests.get(f"https://api.spotify.com/v1/search?q=remaster%2520track%3A{song_title}%2520artist%3A{artist_name}%2520&type=track&limit=1&offset=0", headers=headers)
    if response.status_code == 200:
        return response.json()['tracks']
    else:
        logging.error(f"song_id: {response.status_code}")
        return []
# print(song_id('Brazil', 'Declan McKenna'))


#read in user input song and artist, constraint is that it only returns the most popular/relevant one, get id

#find if it's inside of a playlist



In [25]:
# def get_playlist_tracks(track_title, artist_name):
#     headers = {'Authorization': f"Bearer {access_token}"}
#     response = requests.get(f"https://api.spotify.com/v1/search?q=remaster%2520track%3A{track_title}%2520artist%3A{artist_name}&type=track", headers=headers)
#     if response.status_code == 200:
#         return response.json()['items']
#     else:
#         logging.error(f"get_playlist_tracks: {response.status_code}")
#         return []
